In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import seaborn as sns
color = sns.color_palette()
sns.set_style('darkgrid')

from scipy import stats
from scipy.stats import norm, skew #for some statistics

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)

import warnings
warnings.filterwarnings('ignore')

In [2]:
df_schools = pd.read_csv("Masterlist of Schools.csv", index_col="school.id")

df_location = pd.read_csv("Schools Location Data.csv", 
                          encoding = "latin-1", 
                          index_col="School ID",
                          usecols=["School ID", "Enrolment", "Latitude", "Longitude"])

df_rooms = pd.read_csv('Rooms data.csv', index_col="School ID")

df_teachers = pd.read_csv("Teachers data.csv", index_col="school.id")

df_elementary = pd.read_csv("Enrollment Master Data_2015_E.csv")[:-1].astype(int).set_index("School ID")

df_secondary = (pd.read_csv('Enrollment Master Data_2015_S.csv')[:-1]
                  .replace(",", "", regex=True)
                  .astype(int)
                  .rename(columns = {'SPED NG Male':'SPED NG Male SS', 'SPED NG Female' :'SPED NG Female SS'})
                  .set_index("School ID"))

df_mooe = (pd.read_csv('MOOE data.csv', index_col="school.id", usecols=["school.id", " school.mooe "])
             .replace(",", "", regex=True).astype(float))

FileNotFoundError: [Errno 2] File Masterlist of Schools.csv does not exist: 'Masterlist of Schools.csv'

In [ ]:
df_all = pd.concat([df_schools, df_location, df_rooms, df_teachers, df_elementary, df_secondary, df_mooe], axis=1)
df_all_elementary = pd.concat([df_schools, df_location, df_rooms, df_teachers, df_elementary, df_mooe], axis=1)
df_all_secondary = pd.concat([df_schools, df_location, df_rooms, df_teachers, df_secondary, df_mooe], axis=1)

df_all

In [ ]:
df_secondary = (pd.read_csv('Enrollment Master Data_2015_S.csv')[:-1]
                  .replace(",", "", regex=True)
                  .astype(int)
                  .rename(columns = {'SPED NG Male':'SPED NG Male SS', 'SPED NG Female' :'SPED NG Female SS'})
                  .set_index("School ID"))

df_secondary

In [ ]:
df_all.isna().sum()

In [ ]:
df_all[df_all.index.duplicated(keep=False)]

In [ ]:
df_all.columns

In [ ]:
df_all['Total Elementary SPED Students'] = df_all['SPED NG Male'] + df_all['SPED NG Female']
df_all['Total Secondary SPED Students'] = df_all['SPED NG Male SS'] + df_all['SPED NG Female SS']
df_all['Total Rooms'] = (df_all['rooms.standard.academic'] + 
                             df_all['rooms.standard.unused'] + 
                             df_all['rooms.nonstandard.academic'] + 
                             df_all['rooms.nonstandard.unused'])
df_all['Other Teachers'] = (df_all['teachers.instructor'] + 
                             df_all['teachers.mobile'] + 
                             df_all['teachers.regular'])
                            
df_all['SPED Teachers'] = df_all['teachers.sped']

In [ ]:
df_all.isna().sum()

In [ ]:
df_eda = df_all[['school.classification','school.classification2','school.type','school.urban','Total Elementary SPED Students','Total Secondary SPED Students','Total Rooms','Other Teachers','SPED Teachers',' school.mooe ']]

In [ ]:
corrmat_eda = df_eda.corr()
f, ax = plt.subplots(figsize = (8,6))
sns.heatmap(corrmat_eda, square = True)

The plot shows that: 
<br> 1.) There are more ***SPED Teachers*** in the **Elementary Grade** rather in the **Secondary Grade**.
<br> 2.) ***Other Teachers*** (Regular, Mobile, & Instructor) take much more of the room allocation rather than the *SPED Teachers*. This makes sense because the number of total enrolees for regular students is higher than SPED students.
<br> 3.) The number of ***total rooms*** is highly correlated with the ***school's budget***. This highly suggests that the higher the budget, the higher number of rooms a school can construct/offer.
<br> 4.) Also, a ***school's budget*** has something to do with their capability to hire/have more teachers. Based on the plot, it suggests that the higher the budget of a school, the greater the number of teachers they can deploy.

In [ ]:
df_eda_missing = (df_eda.isnull().sum() / len(df_eda))*100
df_eda_missing = df_eda_missing.drop(df_eda_missing[df_eda_missing == 0].index).sort_values(ascending = False)
missing_data = pd.DataFrame({'Missing Percentage':df_eda_missing})
missing_data

In [ ]:
df_eda.isna().sum()

In [ ]:
df_all.shape

In [ ]:
df_eda_copy = df_eda
df_eda_copy['Total Elementary SPED Students'] = df_eda_copy['Total Elementary SPED Students'].fillna(0)
df_eda_copy['Total Secondary SPED Students'] = df_eda_copy['Total Elementary SPED Students'].fillna(0)
df_eda_copy


In [ ]:
df_eda_missing = (df_eda_copy.isnull().sum() / len(df_eda_copy))*100
df_eda_missing = df_eda_missing.drop(df_eda_missing[df_eda_missing == 0].index).sort_values(ascending = False)
missing_data = pd.DataFrame({'Missing Percentage':df_eda_missing})
missing_data

In [ ]:
corrmat_eda = df_eda_copy.corr()
f, ax = plt.subplots(figsize = (8,6))
sns.heatmap(corrmat_eda, square = True)

The plot shows that: 
<br> 1.) There are more ***SPED Teachers*** in the **Elementary Grade** rather in the **Secondary Grade**.
<br> 2.) ***Other Teachers*** (Regular, Mobile, & Instructor) take much more of the room allocation rather than the *SPED Teachers*. This makes sense because the number of total enrolees for regular students is higher than SPED students.
<br> 3.) The number of ***total rooms*** is highly correlated with the ***school's budget***. This highly suggests that the higher the budget, the higher number of rooms a school can construct/offer.
<br> 4.) Also, a ***school's budget*** has something to do with their capability to hire/have more teachers. Based on the plot, it suggests that the higher the budget of a school, the greater the number of teachers they can deploy.

In [ ]:
df_eda_clean = df_eda_copy.dropna()
df_eda_clean.shape

In [ ]:
df_eda_missing = (df_eda_clean.isnull().sum() / len(df_eda_clean))*100
df_eda_missing = df_eda_missing.drop(df_eda_missing[df_eda_missing == 0].index).sort_values(ascending = False)
missing_data = pd.DataFrame({'Missing Percentage':df_eda_missing})
missing_data

In [ ]:
df_eda_clean.columns

In [ ]:
df_eda_clean['school.urban'].value_counts()

In [ ]:
# Scatterplot all Features
cat_col = ['school.classification', 'school.classification2', 'school.type',
       'school.urban']
for col in cat_col:
    sns.set()
    cols = ['Total Elementary SPED Students', 'Total Secondary SPED Students', 'Total Rooms', 'Other Teachers',
       'SPED Teachers', ' school.mooe ']
    plt.figure()
    sns.pairplot(df_eda_clean, size = 4.0, hue = ['school.classification'])
    plt.show

In [ ]:
sns.pairplot(df_eda_clean, size = 3.0, hue = 'school.classification')

# K-Means Clustering

In [ ]:
df_all['Total Teachers'] = df_all['Other Teachers'] + df_all['SPED Teachers']

df_all['ST Ratio'] = df_all['Enrolment'] / df_all['Total Teachers']

df_all.head()

In [ ]:
df_all['school.classification'].unique()

In [ ]:
df_all_elementary['Total Elementary SPED Students'] = df_all_elementary['SPED NG Male'] + df_all_elementary['SPED NG Female']
df_all_elementary['Total Rooms'] = (df_all_elementary['rooms.standard.academic'] + 
                             df_all_elementary['rooms.standard.unused'] + 
                             df_all_elementary['rooms.nonstandard.academic'] + 
                             df_all_elementary['rooms.nonstandard.unused'])
df_all_elementary['Other Teachers'] = (df_all_elementary['teachers.instructor'] + 
                             df_all_elementary['teachers.mobile'] + 
                             df_all_elementary['teachers.regular'])
                            
df_all_elementary['SPED Teachers'] = df_all_elementary['teachers.sped']

df_all_elementary['Total Elementary SPED Students'] = df_all_elementary['Total Elementary SPED Students'].fillna(0)

df_all_elementary['Total Teachers'] = df_all_elementary['Other Teachers'] + df_all_elementary['SPED Teachers']

df_all_elementary['ST Ratio'] = df_all_elementary['Enrolment'] / df_all_elementary['Total Teachers']

df_all_elementary.head()

In [ ]:
df_all.isna().sum()

In [ ]:
df_all_secondary['Total Secondary SPED Students'] = df_all_secondary['SPED NG Male SS'] + df_all_secondary['SPED NG Female SS']
df_all_secondary['Total Rooms'] = (df_all_secondary['rooms.standard.academic'] + 
                             df_all_secondary['rooms.standard.unused'] + 
                             df_all_secondary['rooms.nonstandard.academic'] + 
                             df_all_secondary['rooms.nonstandard.unused'])
df_all_secondary['Other Teachers'] = (df_all_secondary['teachers.instructor'] + 
                             df_all_secondary['teachers.mobile'] + 
                             df_all_secondary['teachers.regular'])
                            
df_all_secondary['SPED Teachers'] = df_all_secondary['teachers.sped']

df_all_secondary['Total Secondary SPED Students'] = df_all_secondary['Total Secondary SPED Students'].fillna(0)

df_all_secondary['Total Teachers'] = df_all_secondary['Other Teachers'] + df_all_secondary['SPED Teachers']

df_all_secondary['ST Ratio'] = df_all_secondary['Enrolment'] / df_all_secondary['Total Teachers']

df_all_secondary.head()

In [ ]:
df_all.columns

In [ ]:
df_all_elementary.columns

In [ ]:
df_all_secondary.columns

In [ ]:
df_all_kmeans = df_all[['ST Ratio',' school.mooe ','Total Rooms']]
df_all_elementary_kmeans = df_all_elementary[['ST Ratio',' school.mooe ','Total Rooms']]
df_all_secondary_kmeans = df_all_secondary[['ST Ratio',' school.mooe ','Total Rooms']]

df_all_kmeans.replace([np.inf, -np.inf], np.nan, inplace=True)
df_all_elementary_kmeans.replace([np.inf, -np.inf], np.nan, inplace=True)
df_all_secondary_kmeans.replace([np.inf, -np.inf], np.nan, inplace=True)

In [ ]:
df_all_kmeans_clean = df_all_kmeans.dropna()
df_all_elementary_kmeans_clean = df_all_elementary_kmeans.dropna()
df_all_secondary_kmeans_clean = df_all_secondary_kmeans.dropna()

# Elementary

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')

ST_Ratio = df_all_elementary_kmeans_clean['ST Ratio']
Rooms = df_all_elementary_kmeans_clean['Total Rooms']
Budget = df_all_elementary_kmeans_clean[' school.mooe ']
ax.scatter(ST_Ratio, Rooms, Budget, s=5)

ax.set_xlabel('ST Ratio')
ax.set_ylabel('Rooms')
ax.set_zlabel('Budget')

plt.show()

In [ ]:
fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection='3d')

ST_Ratio = df_all_secondary_kmeans_clean['ST Ratio']
Rooms = df_all_secondary_kmeans_clean['Total Rooms']
Budget = df_all_secondary_kmeans_clean[' school.mooe ']
ax.scatter(ST_Ratio, Rooms, Budget, s=5)

ax.set_xlabel('ST Ratio')
ax.set_ylabel('Rooms')
ax.set_zlabel('Budget')

plt.show()

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
df_scaled_ss = scaler.fit_transform(df_ratio_ss)